In [ ]:
import pandas as pd 
import numpy as np
import sys
from google.colab import auth 
import gspread
from google.auth import default
import gspread_dataframe as gd
try: 
  auth.authenticate_user()
  creds, _ = default()
  print('authenticated') 

except: 
  print('authentication failed') 

# setup
gc = gspread.authorize(creds)
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows',None)




class ReadDrive: 
  ## Read Google Sheets Link to be able to read to or write sheets

  def __init__(self, sheet_url): 

    self.gc = gspread.authorize(creds) ## Open the function to be able to read Google Sheets
    self.sheet_url = sheet_url
    self.wb = self.gc.open_by_url(self.sheet_url) 

  def ReadSheet(self, sheet_name): 
    ## Given the sheet name read in the sheet data 
    self.df = pd.DataFrame(self.wb.worksheet(sheet_name).get_all_values()) ## Save as DataFrame
    self.df = pd.DataFrame(self.df.values[1:],columns = self.df.iloc[0]) ## Make the first Column the header, values below
    self.df = self.df.replace(r'^\s*$', np.nan, regex=True) 
    for col in self.df.columns: 
      try: 
        self.df[col] = self.df[col].astype(float)
      except: 
        pass
                           
    return self.df 

  def WriteSheet(self, sheet_name, df): 
    ## Given the sheet name on a spreadsheet write data to this sheet 
    try: 
      sheet = self.wb.worksheet(sheet_name) 
    except: 
      sheet = self.wb.add_worksheet(title = sheet_name, rows = '100', cols = '20') 
    gd.set_with_dataframe(sheet, df)